In [49]:
# Required Packages
try:
    from bs4 import BeautifulSoup
    import requests
    import csv
except ImportError:
    %%capture
    !pip install bs4
    !pip install requests
    from bs4 import BeautifulSoup
    import requests
    import csv

In [44]:
url="https://www.spotrac.com/mlb/payroll/2021/positional"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")
# print(soup.prettify()) # print the parsed data of html

### Find Table of interest on the webpage

In [45]:
mlb_table = soup.find("table", attrs={"class": "datatable rtable captracker"})

mlb_table_header = mlb_table.thead.find_all("tr") # Headers
mlb_table_data = mlb_table.tbody.find_all("tr")  # Rows

### Extract Information from the Table

In [46]:
# Get all the headings
headings = []
for th in mlb_table_header[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(th.text.replace('\n', ' ').strip())

print(headings)

['Rank', 'Team', 'Win %', 'Roster', 'Catcher $', 'Infield $', 'Outfield $', 'DH $', 'Pitching $', 'Total Payroll']


In [47]:
data = []
for tr in mlb_table_data: # find all tr's from table's tbody
    if tr.attrs:
        if tr.attrs["class"][0] == "average":
            continue
    row = {}
    # Each row is stored in the form of
    # row = {'Rank': '', 'Team': '',etc...}

    # find all td's in tr and zip it with headings
    for td, th in zip(tr.find_all("td"), headings):
        row[th] = td.text.replace('\n', '').strip()
    data.append(row)

print(data)

[{'Rank': '1', 'Team': 'Los Angeles Dodgers', 'Win %': '0.654', 'Roster': '28', 'Catcher $': '$2,240,500', 'Infield $': '$48,175,134', 'Outfield $': '$56,600,000', 'DH $': '$426,313', 'Pitching $': '$67,219,595', 'Total Payroll': '$271,200,832'}, {'Rank': '2', 'Team': 'New York Yankees', 'Win %': '0.568', 'Roster': '30', 'Catcher $': '$7,253,767', 'Infield $': '$9,298,409', 'Outfield $': '$13,086,340', 'DH $': '$29,000,000', 'Pitching $': '$82,880,237', 'Total Payroll': '$205,669,863'}, {'Rank': '3', 'Team': 'New York Mets', 'Win %': '0.475', 'Roster': '28', 'Catcher $': '$8,364,690', 'Infield $': '$24,909,097', 'Outfield $': '$26,650,000', 'DH $': '-', 'Pitching $': '$94,641,967', 'Total Payroll': '$201,189,189'}, {'Rank': '4', 'Team': 'Houston Astros', 'Win %': '0.586', 'Roster': '28', 'Catcher $': '$7,000,000', 'Infield $': '$64,952,542', 'Outfield $': '$17,483,098', 'DH $': '$609,000', 'Pitching $': '$57,083,085', 'Total Payroll': '$194,472,041'}, {'Rank': '5', 'Team': 'Philadelphi

### Exporting Data to Excel

In [48]:
with open("mlb_positional.csv", 'w', newline = '') as out_file:
    writer = csv.DictWriter(out_file, headings)
    writer.writeheader()
    writer.writerows(data)